In [1]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

import tensorflow as tf
import pandas as pd
import numpy as np
import sagemaker
import os
from sagemaker.tensorflow import TensorFlow
import time
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from matplotlib import pyplot as plt
import os

2022-11-29 16:44:36.465713: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
input_path = 'data/input/'

In [3]:
all_train_data = pd.read_csv(input_path + 'train_data_prepped.csv').fillna('')
all_test_data = pd.read_csv(input_path + 'test_data_prepped.csv').fillna('')

In [4]:
train_target = pd.read_csv(input_path + 'train_results.csv')
train_target['target'].value_counts()

positive    520436
negative    519803
neutral         84
Name: target, dtype: int64

In [5]:
train_target['target'] = [0 if t == 'negative' else 2 if t == 'positive' else 2 for t in train_target['target'].values]

In [6]:
all_train_data['target'] = train_target['target']

In [7]:
all_train_data.head()

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized,target
0,0,anyway im getting of for a while,train,anyway im getting of for a while,anyway im getting of for a while,anyway im getting,anyway im get,anyway im get,anyway im getting,2
1,1,"my red, apache isn't feelin too well this morn...",train,my red apache isnt feelin too well this mornin...,my red apache isnt feelin too well this mornin...,red apache feelin well morning httpmypictmen,red apach feelin well morn httpmypictmen,red apach feelin wel morn httpmypictmen,red apache feelin well morning httpmypictmen,0
2,2,@danyelljoy you should be its great. friday w...,train,danyelljoy you should be its great friday wil...,danyelljoy you should be its great friday wil...,danyelljoy great friday great tooooooo,danyelljoy great friday great tooooooo,danyelljoy gre friday gre tooooooo,danyelljoy great friday great tooooooo,2
3,3,its 11:30pm and i dont wanna sleep; so i debat...,train,its 1130pm and i dont wanna sleep so i debated...,its pm and i dont wanna sleep so i debated wit...,pm wanna sleep debated end decided perfect tim...,pm wanna sleep debat end decid perfect time ba...,pm wann sleep deb end decid perfect tim bake! kid,pm wanna sleep debated end decided perfect tim...,2
4,4,why does twitter eat my dm's? not happy,train,why does twitter eat my dms ? not happy,why does twitter eat my dms ? not happy,twitter eat dms ? happy,twitter eat dm ? happi,twit eat dms ? happy,twitter eat dm ? happy,0


In [8]:
# all_train_data['text_num_chars'] = [len(text) for text in all_train_data['text'].values]
# all_train_data['text_num_chars'].hist()
# all_train_data['text_num_chars'].describe()

In [9]:
epochs = 40
max_words = 100
max_len = 200
# gpu_count = args.gpu_count
# model_dir = args.model_dir
# training_di = args.train

In [10]:
labels = all_train_data['target'].values
labels = tf.keras.utils.to_categorical(labels, 3, dtype="float32")
features = all_train_data['text_lemmatized'].values
X = []
for i in range(len(features)):
    X.append(str(features[i]))


    

In [11]:
#Tokenizing data and making them sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
features = pad_sequences(sequences, maxlen=max_len)



In [ ]:
vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(features)

In [22]:
features.shape

(1040323, 200)

In [23]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(features[:100_000], labels[:100_000], random_state=0)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
print (len(X_train),len(X_test),len(y_train),len(y_test))
print(type(X_train), type(X_test), type(y_train), type(y_test))

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

75000 25000 75000 25000
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
(75000, 200) (25000, 200) (75000, 3) (25000, 3)


In [24]:
X_train

array([[ 0,  0,  0, ...,  0, 15, 75],
       [ 0,  0,  0, ...,  0,  4, 55],
       [ 0,  0,  0, ..., 56, 61,  5],
       ...,
       [ 0,  0,  0, ...,  0,  0, 40],
       [ 0,  0,  0, ..., 21, 87, 10],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

In [25]:
# Building the model
model = Sequential()
model.add(layers.Embedding(max_words, 40, input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model.add(layers.Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])




2022-11-24 14:37:46.057426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
history = model.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test))

Epoch 1/5
2344/2344 [==============================] - 118s 49ms/step - loss: 0.6315 - accuracy: 0.6241 - val_loss: 0.6048 - val_accuracy: 0.6545
Epoch 2/5
2344/2344 [==============================] - 110s 47ms/step - loss: 0.6111 - accuracy: 0.6452 - val_loss: 0.6032 - val_accuracy: 0.6569
Epoch 3/5
2344/2344 [==============================] - 127s 54ms/step - loss: 0.6104 - accuracy: 0.6455 - val_loss: 0.6024 - val_accuracy: 0.6567
Epoch 4/5
2344/2344 [==============================] - 159s 68ms/step - loss: 0.6100 - accuracy: 0.6462 - val_loss: 0.6023 - val_accuracy: 0.6557
Epoch 5/5
2344/2344 [==============================] - 126s 54ms/step - loss: 0.6099 - accuracy: 0.6466 - val_loss: 0.6021 - val_accuracy: 0.6561


In [51]:
#Validating model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

782/782 - 9s - loss: 0.4968 - accuracy: 0.7555 - 9s/epoch - 11ms/step
Model accuracy:  0.7555199861526489


In [55]:
model.predict(X_test)

782/782 [==============================] - 10s 12ms/step


array([[4.5558226e-01, 1.8222425e-05, 5.4439950e-01],
       [9.4461894e-01, 1.1016793e-06, 5.5379920e-02],
       [3.6494431e-01, 2.3243065e-05, 6.3503242e-01],
       ...,
       [9.2991829e-01, 2.0066780e-06, 7.0079714e-02],
       [4.1700915e-02, 2.0932944e-06, 9.5829701e-01],
       [6.7220174e-02, 3.6591196e-06, 9.3277615e-01]], dtype=float32)

In [56]:
labels = all_train_data['target'].values
labels = tf.keras.utils.to_categorical(labels, 3, dtype="float32")

In [53]:
text_columns = ['text_no_numerals', 'text_no_sw', 'text_porter_stemmed', 'text_lancaster_stemmed', 'text_lemmatized']


In [58]:
for col in text_columns:
#     print(col)
    features = all_train_data[col].values
    X = []
    for i in range(len(features)):
        X.append(str(features[i]))
    
    #Tokenizing data and making them sequences
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    features = pad_sequences(sequences, maxlen=max_len)
    
    #Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(features[:20_000], labels[:20_000], random_state=0)
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    
    # Building the model
    model = Sequential()
    model.add(layers.Embedding(max_words, 40, input_length=max_len))
    model.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
    model.add(layers.Dense(3,activation='softmax'))
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test))


    #Validating model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print(f'column "{col}" Model accuracy: {test_acc}\n\n')


Epoch 1/5
469/469 [==============================] - 34s 64ms/step - loss: 0.6612 - accuracy: 0.5978 - val_loss: 0.5678 - val_accuracy: 0.6996
Epoch 2/5
469/469 [==============================] - 30s 64ms/step - loss: 0.5378 - accuracy: 0.7289 - val_loss: 0.5267 - val_accuracy: 0.7356
Epoch 3/5
469/469 [==============================] - 29s 63ms/step - loss: 0.4955 - accuracy: 0.7618 - val_loss: 0.5214 - val_accuracy: 0.7444
Epoch 4/5
469/469 [==============================] - 29s 63ms/step - loss: 0.4714 - accuracy: 0.7803 - val_loss: 0.5289 - val_accuracy: 0.7392
Epoch 5/5
469/469 [==============================] - 32s 69ms/step - loss: 0.4572 - accuracy: 0.7871 - val_loss: 0.5111 - val_accuracy: 0.7486
157/157 - 2s - loss: 0.5111 - accuracy: 0.7486 - 2s/epoch - 11ms/step
column "text_no_numerals" Model accuracy: 0.7486000061035156
Epoch 1/5
469/469 [==============================] - 36s 69ms/step - loss: 0.6769 - accuracy: 0.5815 - val_loss: 0.5848 - val_accuracy: 0.6918
Epoch 2/5
4

In [59]:
max_words = 10000

In [ ]:
features = all_train_data['text_no_numerals'].values
X = []
for i in range(len(features)):
    X.append(str(features[i]))

#Tokenizing data and making them sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
features = pad_sequences(sequences, maxlen=max_len)

#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(features[:20_000], labels[:20_000], random_state=0)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

# Building the model
model = Sequential()
model.add(layers.Embedding(max_words, 40, input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model.add(layers.Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test))


#Validating model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'column "{col}" Model accuracy: {test_acc}\n\n')

## try matrix representation

In [83]:
max_words=1_000

In [92]:
#Tokenizing data and making them sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_matrix(X, mode="tfidf")
# features = pad_sequences(sequences, maxlen=max_len)

In [93]:
# sequences.shape

In [94]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(sequences[:50_000], labels[:50_000], random_state=0)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
print (len(X_train),len(X_test),len(y_train),len(y_test))
print(type(X_train), type(X_test), type(y_train), type(y_test))

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

37500 12500 37500 12500
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
(37500, 1000) (12500, 1000) (37500, 3) (12500, 3)


In [95]:
# Building the model
model = Sequential()
model.add(layers.Embedding(max_words, 40, input_length=X_train.shape[1]))
model.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model.add(layers.Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])



In [97]:
history = model.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test))

In [ ]:
#Validating model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)